# Lab 4 - Transfer Learning

In this lab, you will developed a custom image classification model to automatically classify the type of land shown in aerial images of 224-meter x 224-meter plots. Land use classification models can be used to track urbanization, deforestation, loss of wetlands, and other major environmental trends using periodically collected aerial imagery. The images used in this lab are based off of imagery from the U.S. National Land Cover Database. U.S. National Land Cover Database defines six primary classes of land use: *Developed*, *Barren*, *Forested*, *Grassland*, *Shrub*, *Cultivated*. Example images from each land use class are shown here:

Developed | Cultivated | Barren
--------- | ------ | ----------
![Developed](https://github.com/jakazmie/images-for-hands-on-labs/raw/master/developed1.png) | ![Cultivated](https://github.com/jakazmie/images-for-hands-on-labs/raw/master/cultivated1.png) | ![Barren](https://github.com/jakazmie/images-for-hands-on-labs/raw/master/barren1.png)

Forested | Grassland | Shrub
---------| ----------| -----
![Forested](https://github.com/jakazmie/images-for-hands-on-labs/raw/master/forest1.png) | ![Grassland](https://github.com/jakazmie/images-for-hands-on-labs/raw/master/grassland1.png) | ![Shrub](https://github.com/jakazmie/images-for-hands-on-labs/raw/master/shrub1.png)

You shall employ a machine learning technique called transfer learning. Transfer learning is one of the fastest (code and run-time-wise) ways to start using deep learning. It allows for the reuse of knowledge gained while solving one problem to a different but related problem. For example, knowledge gained while learning to recognize landmarks and landscapes could apply when trying to recognize aerial land plots. Transfer Learning makes it feasible to train very effective ML models on relatively small training data sets.

Although the primary goal of this lab is to understand how to use Azure ML to orchestrate deep learning workflows rather then to dive into Deep Learning techniques, ask the instructor if you want to better understand the approach utilized in the lab.

You will start by pre-processing training images into a set of powerful features - sometimes referred to as bottleneck features.

To create bottleneck features you will utilize a pre-trained Deep Learning network that was trained on a general computer vision domain. 

Although, the pre-trained network does not know how to classify aerial land plot images, it knows enough about representing image concepts that if we use it to pre-process aerial images, the extracted image features can be used to effectively train a relatively simple classifier on a **limited number** of samples.

The below diagram represents the architecture of our solution.

![Transfer Learning](https://github.com/jakazmie/images-for-hands-on-labs/raw/master/TLArch.png)

We will use **ResNet50** trained on **imagenet** dataset to extract features. We will occasionally refer to this component of the solution as a featurizer. The output of the featurizer is a vector of 2048 floating point numbers, each representing a feature extracted from an image. 

We will then use extracted features to train a simple fully connected neural network (the top) that will peform final image classification.


In [ ]:
# Check core SDK version number
import azureml.core
print("SDK version:", azureml.core.VERSION)

## Connect to AML Workspace

In [ ]:
import azureml.core
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')

## Feature Engineering

The Python script processes an input image dataset into an output bottleneck feature set. The script expects the images to be organized in the below folder structure:
```
Barren/
Cultivated/
Developed/
Forest/
Herbaceous/
Shrub/
```

The location of the input dataset and the location where to save the output dataset are passed to the script as command line parameters. The output dataset will be stored in a binary HDF5 data format used commonly in Machine Learning and High Performance Computing solutions.

The script is designed to work with a large number of images. As such it does not load all input images to memory at once. Instead it utilizes a utility function `load_images` to feed the featurizer. The function yields batches of images - as Numpy arrays - preprocessed to the format required by **ResNet50**. 

We will not attempt to run the script on a full dataset in a local environment. It is very computationally intensive and unless you run it in an evironment equipped with a powerful GPU it would be very slow. 

However, we will demonstrate how to run the script locally using the same small development dataset we used in the previous lab. Running the script locally under the control of Azure ML can be very usefull during development and debugging.

To process the full dataset we will execute the script on a remote Azure ML Compute equipped with NVidia GPU.

### Create a feature engineering script

In [ ]:
import os
script_folder = './script'
os.makedirs(script_folder, exist_ok=True)

In [ ]:
%%writefile $script_folder/extract.py

import os
import numpy as np
import random
import h5py
from tqdm import tqdm

import tensorflow as tf

import azureml.contrib.brainwave.models.utils as utils
from azureml.contrib.brainwave.models import QuantizedResnet50


def get_batch(pathnames, batchsize=64):
    """Yield succesive batches of images"""
    for i in range(0, len(pathnames), batchsize):
        yield pathnames[i:i+batchsize]
        

def load_images(batch):
    """Return a batch of images as a list of bytes sequences"""
    images = []
    for path in batch:
        with open(path, 'rb') as f:
            images.append(f.read())
    return images

def create_bottleneck_features():
    """Createl bottleneck features and save them to H5 formatted file"""
    img_dir = FLAGS.input_data_dir
    
    # Label images 
    
    # Create the dictionary that maps class names into numeric labels   
    label_map = {
        "Barren": 0,
        "Cultivated": 1,
        "Developed": 2,
        "Forest": 3,
        "Herbaceous": 4,
        "Shrub": 5}    

    # Create a list of all images in a root folder with associated numeric labels
    folders = list(label_map.keys())
    labeled_image_list = [(os.path.join(img_dir, folder, image), label_map[folder]) 
                          for folder in folders 
                          for image in os.listdir(os.path.join(img_dir, folder))
                              ]
    # Shuffle the list
    random.shuffle(labeled_image_list)
    image_paths, labels = zip(*labeled_image_list)
    
    # Build featurizer graph
    
    # Convert input images (loaded as bytes sequences) into (224, 224, 3) tensors
    # with pixel values in Caffe encoding
    in_images = tf.placeholder(tf.string)
    image_tensors = utils.preprocess_array(in_images)

    # Create ResNet152 
    model_path = os.path.expanduser('~/models')
    resnet = QuantizedResnet50(model_path, is_frozen=True)

    # Import ResNet152 graph
    features = resnet.import_graph_def(input_tensor=image_tensors)
    
    # Generate bottleneck features
    print("Generating bottleneck features")
    bottleneck_features = []
    with tf.Session() as sess:
        for paths in tqdm(get_batch(image_paths)):
            image_batch = load_images(paths)
            result = sess.run([features], feed_dict={in_images: image_batch})
            result = np.reshape(result[0], (len(result[0]), 2048))
            bottleneck_features.extend(result)
        
    bottleneck_features = np.array(bottleneck_features)
    print(bottleneck_features.shape)
        
    # Save the bottleneck features to HDF5 file
    filename = FLAGS.file_name
    output_file = os.path.join(FLAGS.output_data_dir, filename)
    labels = np.asarray(labels)
    print("Saving bottleneck features to {}".format(output_file))
    print("   Features: ", bottleneck_features.shape)
    print("   Labels: ", labels.shape)
    with h5py.File(output_file, "w") as hfile:
        features_dset = hfile.create_dataset('features', data=bottleneck_features)
        labels_dset = hfile.create_dataset('labels', data=labels)
    
    print("Done")

FLAGS = tf.app.flags.FLAGS

# Default global parameters
tf.app.flags.DEFINE_integer('batch_size', 64, "Number of images per batch")
tf.app.flags.DEFINE_string('input_data_dir', 'aerialtiny', "Folder with training and validation images")
tf.app.flags.DEFINE_string('output_data_dir', 'bottleneck_features', "A folder for saving bottleneck features")
tf.app.flags.DEFINE_string('file_name', 'aerial_bottleneck_resnet50.h5', "Name of output training file")


def main(argv=None):
    print("Starting")
    print("Reading images from:", FLAGS.input_data_dir)
    print("The output bottleneck file will be saved to:", FLAGS.output_data_dir)

    os.makedirs(FLAGS.output_data_dir, exist_ok=True)

    create_bottleneck_features()
  
if __name__ == '__main__':
    tf.app.run()

### Create Azure ML Managed Compute

We will use an autoscale cluster of *Standard_NC6* VMs (equipped with Tesla K80 GPU). 

In [ ]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
import os


# choose a name for your cluster
compute_name = os.environ.get("AML_COMPUTE_CLUSTER_NAME", "gpu-bai-cluster")
compute_min_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MIN_NODES", 1)
compute_max_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MAX_NODES", 4)

vm_size = os.environ.get("AML_COMPUTE_CLUSTER_SKU", "STANDARD_NC6")

if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print('found compute target. just use it. ' + compute_name)
else:
    print('creating a new compute target...')
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = vm_size,
                                                                min_nodes = compute_min_nodes, 
                                                                max_nodes = compute_max_nodes)

    # create the cluster
    compute_target = ComputeTarget.create(ws, compute_name, provisioning_config)

    # can poll for a minimum number of nodes and for a specific timeout. 
    # if no min node count is provided it will use the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

     # For a more detailed view of current AmlCompute status, use the 'status' property    
    print(compute_target.status.serialize())

### Configure Datastores 
The training images have been uploaded to a public Azure blob storage container. We will register this container as an AML Datastore within our workspace. Before the data prep script runs, the datastore's content - training images - will be copied to the local storage on the compute nodes.

After the script completes, its output - the bottleneck features file - will be uploaded by AML to the workspace's default datastore.

In [ ]:
from azureml.core import Datastore

images_account = 'azureailabs'
images_container = 'aerial-small'
datastore_name = 'input_images'

# Check if the datastore exists. If not create a new one
try:
    input_ds = Datastore.get(ws, datastore_name)
    print('Found existing datastore for input images:', input_ds.name)
except:
    input_ds = Datastore.register_azure_blob_container(workspace=ws, datastore_name=datastore_name,
                                            container_name=images_container,
                                            account_name=images_account)
    print('Creating new datastore for input images')

 
   
print(input_ds.name, input_ds.datastore_type, input_ds.account_name, input_ds.container_name)

output_ds = ws.get_default_datastore()
print("Using the default datastore for output: ")
print(output_ds.name, output_ds.datastore_type, output_ds.account_name, output_ds.container_name)


### Create AML Experiment
We will track runs of the feature engineering script in a dedicated Experiment.

In [ ]:
from azureml.core import Experiment
experiment_name = 'aerial-feature-engineering'
exp = Experiment(workspace=ws, name=experiment_name)

### Start and monitor a remote run

We will run a script on a single node in a docker container. The docker image will be configured and created using AML APIs.

The first run takes longer. The subsequent runs, as long as the script dependencies don't change, are much faster.

You can check the progress of a running job in multiple ways: Azure Portal, AML Jupyter Widgets, log files streaming. We will use AML Jupyter Widgets.

In [ ]:
from azureml.train.estimator import Estimator

# Define the location of the dataprep script and the location for the output bottleneck files
script_folder = 'script'
script_name = 'extract.py'
output_dir = 'bottleneck_features'

pip_packages = ['h5py', 
                'pillow', 
                'tqdm', 
                'azureml-sdk[contrib]', 
                'scikit-learn', 
                'tensorflow-gpu==1.10']

script_params = {
    '--input_data_dir': input_ds.as_download(),
    '--output_data_dir': output_dir,
    '--file_name': 'aerial_bottleneck_resnet50_brainwave.h5'
}

est = Estimator(source_directory=script_folder,
                script_params=script_params,
                compute_target=compute_target,
                entry_script=script_name,
                node_count=1,
                process_count_per_node=1,
                use_gpu=True,
                pip_packages=pip_packages,
                inputs=[output_ds.path(output_dir).as_upload(path_on_compute=output_dir)])

Submit the run and start RunDetails widget.

In [ ]:
from azureml.widgets import RunDetails

tags = {"Compute target": "AML Compute GPU", "DNN": "Brainwave ResNet50"}
run = exp.submit(config=est, tags=tags)

RunDetails(run).show()

Block to wait till the run finishes.

In [ ]:
run.wait_for_completion(show_output=False)

After the run, AML copied the output bottleneck files to the default datastore. You can verify it using Azure Portal.

## Training
The run has completed. The next step is to train a small Fully Connected Neural Network using engineered bottleneck features.

We will use AML feature called `Hyperdrive` to fine tune hyperparameters of the neural network. `Hyperdrive` will utilize Azure ML Compute GPU cluster to run and evaluate concurrent training jobs. After the model is fine tuned, the best version will be registered in AML Model Registry.

### Create training script

In the training script, we use Tensorflow.Keras to define and train a simple fully connected neural network.

The network has one hidden layer. The input to the network is a vector of 2048 floating point numbers - the bottleneck features created in the previous step. The output layer consists of 6 units - representing six land type classes. To control overfitting the network uses a Dropout layer between the hidden layer and the output layer and L1 and L2 regularization in the output layer.

The number of units in the hidden layer, L1 and L2 values, and batch size are all tuneable hyperparameters. The Dropout ratio is fixed at 0.5.

Since the bottleneck feature files are small (as compared to original image datasets) they can be loaded into memory all at once.

The trained model will be saved into the ./outputs folder. This is one of the special folders in AML. The other one is the ./logs folder. The content in these folders is automatically uploaded to the run history.

The script uses AML Run object to track two performane measures: training accuracy and validation accuracy. The metrics are captured at the end of each epoch.


In [ ]:
script_name = 'train.py'

In [ ]:
%%writefile $script_folder/train.py

import os
import tensorflow as tf
from tensorflow.keras.applications import resnet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Flatten, Input
from tensorflow.keras.regularizers import l1_l2
from sklearn.model_selection import train_test_split

from azureml.core import Run

import numpy as np
import random
import h5py


# Create custom callback to track accuracy measures in AML Experiment
class RunCallback(tf.keras.callbacks.Callback):
    def __init__(self, run):
        self.run = run
        
    def on_epoch_end(self, batch, logs={}):
        self.run.log(name="training_acc", value=float(logs.get('acc')))
        self.run.log(name="validation_acc", value=float(logs.get('val_acc')))


# Define network
def fcn_classifier(input_shape=(2048,), units=512, classes=6,  l1=0.01, l2=0.01):
    features = Input(shape=input_shape)
    x = Dense(units, activation='relu')(features)
    x = Dropout(0.5)(x)
    y = Dense(classes, activation='softmax', kernel_regularizer=l1_l2(l1=l1, l2=l2))(x)
    model = Model(inputs=features, outputs=y)
    model.compile(optimizer='adadelta', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Training regime
def train_evaluate(run):
   
    print("Loading bottleneck features")
    train_file_name = os.path.join(FLAGS.data_folder, FLAGS.train_file_name)
    
    # Load bottleneck training features and labels
    with h5py.File(train_file_name, "r") as hfile:
        features = np.array(hfile.get('features'))
        labels = np.array(hfile.get('labels'))
        
    
        
    # Split the data into training and validation partitions   
    X_train, X_validation, y_train, y_validation = train_test_split(features, labels,
                                                               test_size=0.1,
                                                               shuffle=True,
                                                               stratify=labels)
        
    # Convert labels into one-hot encoded format
    y_train = to_categorical(y_train, num_classes=6)
    y_validation = to_categorical(y_validation, num_classes=6)
    
    # Create a network
    model = fcn_classifier(input_shape=(2048,), units=FLAGS.units, l1=FLAGS.l1, l2=FLAGS.l2)
    
    # Create AML tracking callback
    run_callback = RunCallback(run)
    
    # Start training
    print("Starting training")
    model.fit(X_train, y_train,
          batch_size=FLAGS.batch_size,
          epochs=FLAGS.epochs,
          shuffle=True,
          validation_data=(X_validation, y_validation),
          callbacks=[run_callback])
          
    # Save the trained model to outputs which is a standard folder expected by AML
    print("Training completed.")
    os.makedirs('outputs', exist_ok=True)
    model_file = os.path.join('outputs', 'aerial_fcnn_classifier.hd5')
    print("Saving model to: {0}".format(model_file))
    model.save(model_file)
    

FLAGS = tf.app.flags.FLAGS

# Default global parameters
tf.app.flags.DEFINE_integer('batch_size', 32, "Number of images per batch")
tf.app.flags.DEFINE_integer('epochs', 10, "Number of epochs to train")
tf.app.flags.DEFINE_integer('units', 512, "Number of epochs to train")
tf.app.flags.DEFINE_float('l1', 0.01, "l1 regularization")
tf.app.flags.DEFINE_float('l2', 0.01, "l2 regularization")
tf.app.flags.DEFINE_string('data_folder', './bottleneck', "Folder with bottleneck features and labels")
tf.app.flags.DEFINE_string('train_file_name', 'aerial_bottleneck_resnet50.h5', "Training file name")

def main(argv=None):
    
    # get hold of the current run
    run = Run.get_submitted_run()
    train_evaluate(run)
  

if __name__ == '__main__':
    tf.app.run()
    

### Configure datastore

The bottleneck files have been uploaded to the workspace's default datastore during the previous step. We will mount the store on the nodes of the cluster.


In [ ]:
from azureml.core import Datastore

ds = ws.get_default_datastore()
print("Using the default datastore for training data: ")
print(ds.name, ds.datastore_type, ds.account_name, ds.container_name)


### Run a test run on a single node of the cluster

In [ ]:
from azureml.train.estimator import Estimator

script_params = {
    '--data_folder': ds.path('bottleneck_features').as_download(),
    '--train_file_name': 'aerial_bottleneck_resnet50_brainwave.h5',
    '--l1': 0.001,
    '--l2': 0.001,
    '--units': 512,
    '--epochs': 10
}


#pip_packages = ['h5py','pillow', 'scikit-learn', 'tensorflow-gpu==1.10']

pip_packages = ['h5py', 
                'pillow', 
                'tqdm', 
                'azureml-sdk[contrib]', 
                'scikit-learn', 
                'tensorflow-gpu==1.10']

est = Estimator(source_directory=script_folder,
                script_params=script_params,
                compute_target=compute_target,
                entry_script=script_name,
                node_count=1,
                process_count_per_node=1,
                use_gpu=True,
                pip_packages=pip_packages,
                )


In [ ]:
tags = {"Compute target": "BAI", "Run Type": "Test drive"}
run = exp.submit(est, tags=tags)
run

In [ ]:
from azureml.widgets import RunDetails
RunDetails(run).show()

In [ ]:
run.wait_for_completion(show_output=True)

### Configure `Hyperdrive`

As noted before, our network has 5 hyperparameters:

- Number of units in the hidden layer
- L1 and L2 regularization
- mini-batch size, and
- dropout ratio

As we have limited time to complete the lab, we are going to limit a number of hyperparameter combinations to try. We will use a fixed batch-size and dropout ratio and focus on hidden layer units and L1 and L2 regularization.

*Hyperdrive* supports many strategies for sampling the hyperparameter space. In this lab, we are going to use the simplest one - grid sampling.

In [ ]:
from azureml.train.hyperdrive import *

ps = GridParameterSampling(
    {
        '--units': choice(256, 512),
        '--l1': choice(0.001, 0.01, 0.05),
        '--l2': choice(0.001, 0.01, 0.05)
    }
)

We will use **Estimator** object to configure the training job. Note how we pass the location of the bottleneck files to the estimator. The job will run on GPU VMs and as such we need to use the GPU version of Tensorflow.

In [ ]:
from azureml.train.estimator import Estimator

script_params = {
    '--data_folder': ds.path('bottleneck_features').as_download(),
    '--train_file_name': 'aerial_bottleneck_resnet50_brainwave.h5',
    '--epochs': 50
}

#pip_packages = ['h5py','pillow', 'scikit-learn', 'tensorflow-gpu']

pip_packages = ['h5py', 
                'pillow', 
                'tqdm', 
                'azureml-sdk[contrib]', 
                'scikit-learn', 
                'tensorflow-gpu==1.10']


est = Estimator(source_directory=script_folder,
                script_params=script_params,
                compute_target=compute_target,
                entry_script=script_name,
                node_count=1,
                process_count_per_node=1,
                use_gpu=True,
                pip_packages=pip_packages,
                )


*Hyperdrive* supports early termination policies to limit exploration of hyperparameter combinations that don't show promise of helping reach the target metric. This feature is especially useful when traversing large hyperparameter spaces. Since we are going to run a small number of jobs we will not apply early termination.

In [ ]:
policy = NoTerminationPolicy()

Now we are ready to configure a run configuration object, and specify the primary metric as *validation_acc* that's recorded in our training runs. If you go back to visit the training script, you will notice that this value is being logged after every run. We also want to tell the service that we are looking to maximizing this value. We also set the number of total runs to 12, and maximal concurrent job to 4, which is the same as the number of nodes in our computer cluster. 

In [ ]:
htc = HyperDriveRunConfig(estimator=est, 
                          hyperparameter_sampling=ps,
                          policy=policy,
                          primary_metric_name='validation_acc', 
                          primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
                          max_total_runs=12,
                          max_concurrent_runs=4)

Finally, let's launch the hyperparameter tuning job.

The first run takes longer as the system has to prepare and deploy a docker image with training job runtime dependencies. As long as the dependencies don't change the following runs will be much faster.

Here is what's happening whie you wait.

- **Image creation**: A Docker image is created matching the Python environment specified by the estimator. The image is uploaded to the workspace. This stage happens once for each Python environment since the container is cached for subsequent runs. During image creation, logs are streamed to the run history. You can monitor the image creation progress using these logs.

- **Scaling**: If the remote cluster requires more nodes to execute the run than currently available, additional nodes are added automatically.

- **Running**: In this stage, the necessary scripts and files are sent to the compute target, then data stores are mounted/copied, then the entry_script is run. While the job is running, stdout and the ./logs directory are streamed to the run history. You can monitor the run's progress using these logs.

- **Post-Processing**: The ./outputs directory of the run is copied over to the run history in your workspace so you can access these results.


In [ ]:
tags = {"Training": "Hyperdrive"}

hdr = exp.submit(config=htc, tags=tags)
hdr

In [ ]:
from azureml.widgets import RunDetails
RunDetails(hdr).show()

In [ ]:
hdr.wait_for_completion(show_output=False) # specify True for a verbose log

### Find and register best model
When all jobs finish, we can find out the one that has the highest accuracy.

In [ ]:
best_run = hdr.get_best_run_by_primary_metric()

In [ ]:
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['Arguments']

print('Best Run Id: ', best_run.id)
print('\n Validation Accuracy:', best_run_metrics['validation_acc'])
print('\n Units:',parameter_values[7])
print('\n L1:',parameter_values[9])
print('\n L2:',parameter_values[11])

Check the output of the best run.

In [ ]:
print(best_run.get_file_names())

### Register model
The last step in the training script wrote the file `aerial_fcnn_classifier.hd5` in the `outputs` directory. As noted before, `outputs` is a special directory in that all content in this  directory is automatically uploaded to your workspace.  This content appears in the run record in the experiment under your workspace. 

You can register the model so that it can be later queried, examined and deployed.

In [ ]:
model = best_run.register_model(model_name='aerial_classifier', 
                                model_path='outputs/aerial_fcnn_classifier.hd5')
print(model.name, model.id, model.version, sep = '\t')

## Clean up resources
Before you move to the next step, delete the cluster.

In [ ]:
compute_target.delete()